In [1]:
import pandas as pd
from mxnet import gluon, autograd, init, nd
from mxnet.gluon import nn
import random
import matplotlib.pyplot as plt

In [2]:
dfraw = pd.read_csv('../data/FinalData.csv')
dfc = dfraw.drop_duplicates()
assert len(dfc) == len(dfraw)
len(dfc)

151256

In [3]:
dfc.columns

Index(['SerialNumber', 'City', 'Name', 'FirstOnline', 'Datetime', 'PM2.5',
       'USAQI', 'CO2', 'Temperature', 'RelativeHumidity', 'Outdoor PM2.5',
       'OutdoorUSAQI', 'weekday', 'weekdayName', 'hour', 'month', 'year',
       'dayofmonth', 'tmpc', 'relh', 'drct', 'sped', 'vsby'],
      dtype='object')

In [4]:
drop_columns = ['City', 'Name', 'FirstOnline', 'Outdoor PM2.5', 'OutdoorUSAQI', 'weekdayName']
df = dfc.drop(drop_columns, axis = 1)

In [5]:
df.isna().mean() * 100

SerialNumber         9.891178
Datetime             0.000000
PM2.5                0.000000
USAQI                0.000000
CO2                  9.891178
Temperature          0.991035
RelativeHumidity     0.991035
weekday              0.000000
hour                 0.000000
month                0.000000
year                 0.000000
dayofmonth           0.000000
tmpc                22.101603
relh                22.483736
drct                22.043423
sped                21.995161
vsby                21.975327
dtype: float64

In [6]:
df= df.dropna()
len(df)

110417

In [ ]:
df['Datetime'] = pd.to_datetime(df.Datetime)

In [ ]:
dfs = df.sort_values(['SerialNumber', 'Datetime'])
dfs = dfs.reset_index(drop = True)

In [ ]:
batch_size = 16

In [ ]:
def make_dataset_many_to_one(dataframe, time_steps):
    rows = dataframe.shape[0] - time_steps
    cols = dataframe.shape[1]
    dfnew = nd.zeros(shape = (rows, cols * time_steps))
    for i in range(0, time_steps):
        dfnew[:, i * cols : (i + 1 ) * cols] = dataframe.iloc[i: i + rows,:]
    ynew = nd.zeros(shape = (rows, 1))
    ynew[:,0] = dataframe['PM2.5'].iloc[time_steps:]
    return dfnew, ynew

In [ ]:
def myPartSerial(i, time_steps):
    dfslice = dfs[dfs.SerialNumber == i]
    lis = []
        
    dfdiff = dfslice.Datetime.diff()
    dfdiff.iloc[0] = dfdiff.iloc[1]
    dfdiff2 = dfdiff != pd.Timedelta('1 hour')
    dfdiff3 = dfdiff2.cumsum()    
    
    for j,k in dfslice.groupby(dfdiff3):
        if len(k) > time_steps:
            d = k.iloc[:]
            drop_columns2 = ['SerialNumber', 'Datetime', 'year', 'relh', 'tmpc']
            d = d.drop(drop_columns2, axis = 1)
            lis.append( d )
    
    newlis = []
    newlabels = []
    for i in lis:
        a, b = make_dataset_many_to_one(i, time_steps)
        assert a.shape[0] == b.shape[0]
        newlis.append(a)
        newlabels.append(b)
    
        
#    if lis:
#        lis = pd.concat(lis)
#    else:
#        lis = dfslice.iloc[0:0]
    
    newlis = nd.array(newlis)
    print(newlist.shape)
    
    
    return newlis, newlabels

In [ ]:
dfs.head(5)

In [ ]:
#lis = []
for i in dfs.SerialNumber.unique()[:1]:
    li = myPartSerial(i, 4)
#    lis.append(li)

#lis = pd.concat(lis)
#print(len(lis))

In [ ]:
print(dfs.SerialNumber.unique())

In [ ]:
net = nn.Sequential()
net.add(gluon.rnn.LSTM(100, num_layers = 3),
       nn.Dense(1)
       )

net